In [1]:
import torchaudio
import pandas as pd
import numpy as np

/home/akansh/anaconda3/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [2]:
import os
import json  
from tqdm import tqdm 

In [3]:
def labels_eko_to_mic (wav_file_name):
    b = k.split('_')
    b[1] = 'Mic'
    separator = '_'
    return separator.join(b)[:-3] +'wav'
    

In [4]:
def MFCC(waveform, sample_rate):
    mfcc = torchaudio.compliance.kaldi.mfcc(waveform=waveform,num_ceps = 12,   
                                     cepstral_lifter=22, channel=0, 
                                     dither = 0.0, energy_floor=0.0,
                                     frame_length=20, frame_shift=10, 
                                     high_freq=sample_rate/2,low_freq = 40, num_mel_bins=25, 
                                     sample_frequency=sample_rate, use_energy=True, vtln_low =60  , vtln_high = 7200, 
                                     window_type="hamming")
    return mfcc

In [5]:
def labels_eko (wav_file_name):
    k = wav_file_name
    b = k.split('_')
    b[1] = 'Label'
    separator = '_'
    return separator.join(b)[:-3] +'txt'
    

In [6]:
set_loc = "./Sets/"
data_loc = "./"
new_sample_rate = 16000
channel = 0
for i in os.listdir(set_loc):
    set_full = np.loadtxt(set_loc + i, dtype=str)
    for j in tqdm(set_full):
        A = {}
        eko = {}
        mic = {}
#         eko = []
#         mic = []
        B_R_T_DPB = []
        B_R_D_DPB = []
        B_L_D_DPB = []
        B_L_T_DPB = []
        for k in os.listdir(data_loc+j+'/Eko'):
            sub_wav_mfcc = []
            waveform_eko, sample_rate = torchaudio.load(data_loc+j+'/Eko/'+k)
            txt_file = pd.read_csv(data_loc+j+'/labels/'+labels_eko(k), sep="\t", header = None)
            labels = txt_file.iloc[:,2]
            start = txt_file.iloc[:,0]
            stop = txt_file.iloc[:,1]
            for l,m in enumerate(labels):
                if m.find('nnotat') != -1:
                    sub_wav_mfcc.append(MFCC(waveform_eko[:,int(round(start[l]*sample_rate)):int(round(stop[l]*sample_rate))], sample_rate).numpy().tolist())
            if k.find('B_R_T_DPB') != -1:
                B_R_T_DPB.append(sub_wav_mfcc)
            if k.find('B_R_D_DPB') != -1:
                B_R_D_DPB.append(sub_wav_mfcc)
            if k.find('B_L_D_DPB') != -1:
                B_L_D_DPB.append(sub_wav_mfcc)
            if k.find('B_L_T_DPB') != -1:
                B_L_T_DPB.append(sub_wav_mfcc)
        eko['B_R_T_DPB'] = B_R_T_DPB
        eko['B_R_D_DPB'] = B_R_D_DPB
        eko['B_L_D_DPB'] = B_L_D_DPB
        eko['B_L_T_DPB'] = B_L_T_DPB
        A['eko']= eko
        B_R_T_DPB = []
        B_R_D_DPB = []
        B_L_D_DPB = []
        B_L_T_DPB = []
        for k in os.listdir(data_loc+j+'/Mic'):
            sub_wav_mfcc = []
            waveform_mic, sample_rate = torchaudio.load(data_loc+j+'/Mic/'+k)
            transformed = torchaudio.transforms.Resample(sample_rate, new_sample_rate)(waveform_mic[channel,:].view(1,-1))
            txt_file = pd.read_csv(data_loc+j+'/labels/'+labels_eko(k), sep="\t", header = None)
            labels = txt_file.iloc[:,2]
            start = txt_file.iloc[:,0]
            stop = txt_file.iloc[:,1]
            for l,m in enumerate(labels):
                if m.find('nnotat') != -1:
                    sub_wav_mfcc.append(MFCC(transformed[:,int(round(start[l]*new_sample_rate)):int(round(stop[l]*new_sample_rate))], new_sample_rate).numpy().tolist())
            if k.find('B_R_T_DPB') != -1:
                B_R_T_DPB.append(sub_wav_mfcc)
            if k.find('B_R_D_DPB') != -1:
                B_R_D_DPB.append(sub_wav_mfcc)
            if k.find('B_L_D_DPB') != -1:
                B_L_D_DPB.append(sub_wav_mfcc)
            if k.find('B_L_T_DPB') != -1:
                B_L_T_DPB.append(sub_wav_mfcc)        
        mic['B_R_T_DPB'] = B_R_T_DPB
        mic['B_R_D_DPB'] = B_R_D_DPB
        mic['B_L_D_DPB'] = B_L_D_DPB
        mic['B_L_T_DPB'] = B_L_T_DPB
        A['mic']= mic
        with open("./"+j+".json", "w") as outfile:  
            json.dump(A, outfile)
        
            







  0%|          | 0/7 [00:00<?, ?it/s]/home/akansh/anaconda3/lib/python3.7/site-packages/torchaudio/compliance/kaldi.py:574: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /pytorch/aten/src/ATen/native/SpectralOps.cpp:590.)
  fft = torch.rfft(strided_input, 1, normalized=False, onesided=True)
100%|██████████| 7/7 [01:03<00:00,  9.12s/it]


In [9]:
waveform_mic, sample_rate = torchaudio.load("./Anita_Mic_2019_11_06_12_53_28.270_B_R_T_DPB_1.wav")

In [24]:
waveform_mic.shape[1]/sample_rate

45.506530612244894

In [17]:
channel = 0
new_sample_rate = 16000
transformed = torchaudio.transforms.Resample(sample_rate, new_sample_rate)(waveform_mic[channel,:].view(1,-1))

tensor([-0.0133, -0.0113, -0.0119,  ..., -0.0014, -0.0010, -0.0017])

### 144 MFCC(concat eko and Mic)

In [18]:
set_loc = "./Sets/"
data_loc = "./"
for i in os.listdir(set_loc):
    set_full = np.loadtxt(set_loc + i, dtype=str)
    for j in tqdm(set_full):
        A = {}
        eko = []
        mic = []   
        for k in os.listdir(data_loc+j+'/Eko'):
            sub_wav_mfcc_eko = []
            sub_wav_mfcc_mic = []
            waveform_eko, sample_rate = torchaudio.load(data_loc+j+'/Eko/'+k)
            waveform_mic, sample_rate_2 = torchaudio.load(data_loc+j+'/Mic/'+labels_eko_to_mic(k))
            txt_file = pd.read_csv(data_loc+j+'/labels/'+labels_eko(k), sep="\t", header = None)
            labels = txt_file.iloc[:,2]
            start = txt_file.iloc[:,0]
            stop = txt_file.iloc[:,1]
            for l,m in enumerate(labels):
                if m.find('nnotat') != -1:
                    sub_wav_mfcc_eko.append(MFCC(waveform_eko[:,int(round(start[l]*sample_rate)):int(round(stop[l]*sample_rate))], sample_rate).numpy().tolist())
                    sub_wav_mfcc_mic.append(MFCC(waveform_mic[:,int(round(start[l]*sample_rate_2)):int(round(stop[l]*sample_rate_2))], sample_rate_2).numpy().tolist())
            eko.append(sub_wav_mfcc_eko)
            mic.append(sub_wav_mfcc_mic)
            
        
        
        
        
        
        A['eko']= eko
        A['mic']= mic
        with open("./144_MFCC_calculated/"+j+".json", "w") as outfile:  
            json.dump(A, outfile)
       
    
    
    
#     for k in os.listdir(data_loc+j+'/Mic'):
#             sub_wav_mfcc = []
#             waveform_mic, sample_rate = torchaudio.load(data_loc+j+'/Mic/'+k)
#             txt_file = pd.read_csv(data_loc+j+'/labels/'+labels_eko(k), sep="\t", header = None)
#             labels = txt_file.iloc[:,2]
#             start = txt_file.iloc[:,0]
#             stop = txt_file.iloc[:,1]
#             for l,m in enumerate(labels):
#                 if m.find('nnotat') != -1:
#                     sub_wav_mfcc.append(MFCC(waveform_mic[:,int(round(start[l]*sample_rate)):int(round(stop[l]*sample_rate))], sample_rate).numpy().tolist())
#             mic.append(sub_wav_mfcc)
#         A['mic']= mic
#         with open("./MFCC_calculated/"+j+".json", "w") as outfile:  
#             json.dump(A, outfile)
        
            







100%|██████████| 7/7 [00:38<00:00,  5.68s/it]


### MFCC Calculate for Aug_data

In [8]:
label_loc = "./"

In [17]:
def labels_mic(wav_file_name):
    if wav_file_name[0] == 'r':
        return labels_eko(wav_file_name)[6:]
    if wav_file_name[0] == 'u':
        return labels_eko(wav_file_name[21:])
    if wav_file_name[0] == 'b':
        return labels_eko(wav_file_name[12:])
    

In [22]:
set_loc = "./Sets/"
data_loc = "./Augmented_Data/Aug"
label_loc = "./"
new_sample_rate = 16000
channel = 0
for i in os.listdir(set_loc):
    set_full = np.loadtxt(set_loc + i, dtype=str)
    for j in tqdm(set_full):
        A = {}
        eko = {}
        mic = {}
        B_R_T_DPB = []
        B_R_D_DPB = []
        B_L_D_DPB = []
        B_L_T_DPB = []
        for k in os.listdir(data_loc+j+'/Eko'):
            sub_wav_mfcc = []
            waveform_eko, sample_rate = torchaudio.load(data_loc+j+'/Eko/'+k)
            txt_file = pd.read_csv(label_loc+j+'/labels/'+labels_eko(k)[6:], sep="\t", header = None)
            labels = txt_file.iloc[:,2]
            start = txt_file.iloc[:,0]
            stop = txt_file.iloc[:,1]
            for l,m in enumerate(labels):
                if m.find('nnotat') != -1:
                    sub_wav_mfcc.append(MFCC(waveform_eko[:,int(round(start[l]*sample_rate)):int(round(stop[l]*sample_rate))], sample_rate).numpy().tolist())
            if k.find('B_R_T_DPB') != -1:
                B_R_T_DPB.append(sub_wav_mfcc)
            if k.find('B_R_D_DPB') != -1:
                B_R_D_DPB.append(sub_wav_mfcc)
            if k.find('B_L_D_DPB') != -1:
                B_L_D_DPB.append(sub_wav_mfcc)
            if k.find('B_L_T_DPB') != -1:
                B_L_T_DPB.append(sub_wav_mfcc)
        eko['B_R_T_DPB'] = B_R_T_DPB
        eko['B_R_D_DPB'] = B_R_D_DPB
        eko['B_L_D_DPB'] = B_L_D_DPB
        eko['B_L_T_DPB'] = B_L_T_DPB
        A['eko']= eko
        B_R_T_DPB = []
        B_R_D_DPB = []
        B_L_D_DPB = []
        B_L_T_DPB = []
        for k in os.listdir(data_loc+j+'/Mic'):
            sub_wav_mfcc = []
            waveform_mic, sample_rate = torchaudio.load(data_loc+j+'/Mic/'+k)
            transformed = torchaudio.transforms.Resample(sample_rate, new_sample_rate)(waveform_mic[channel,:].view(1,-1))
            txt_file = pd.read_csv(label_loc+j+'/labels/'+labels_mic(k), sep="\t", header = None)
            labels = txt_file.iloc[:,2]
            start = txt_file.iloc[:,0]
            stop = txt_file.iloc[:,1]
            for l,m in enumerate(labels):
                if m.find('nnotat') != -1:
                    sub_wav_mfcc.append(MFCC(transformed[:,int(round(start[l]*new_sample_rate)):int(round(stop[l]*new_sample_rate))], new_sample_rate).numpy().tolist())
            if k.find('B_R_T_DPB') != -1:
                B_R_T_DPB.append(sub_wav_mfcc)
            if k.find('B_R_D_DPB') != -1:
                B_R_D_DPB.append(sub_wav_mfcc)
            if k.find('B_L_D_DPB') != -1:
                B_L_D_DPB.append(sub_wav_mfcc)
            if k.find('B_L_T_DPB') != -1:
                B_L_T_DPB.append(sub_wav_mfcc)        
        mic['B_R_T_DPB'] = B_R_T_DPB
        mic['B_R_D_DPB'] = B_R_D_DPB
        mic['B_L_D_DPB'] = B_L_D_DPB
        mic['B_L_T_DPB'] = B_L_T_DPB
        A['mic']= mic
        with open("./"+'Aug'+j+".json", "w") as outfile:  
            json.dump(A, outfile)
        
            
        
        
        

100%|██████████| 7/7 [00:42<00:00,  6.85s/it]
